# Homework Assignment no. 2

## Introduction
1. Import modules
2. Import module with solutions
3. Read csv data

In [1]:
import numpy as np
import pandas as pd
from Zalewski_Jacek_assignment_2 import *

In [2]:
Posts = pd.read_csv("travel_stackexchange_com/Posts.csv")
Badges = pd.read_csv("travel_stackexchange_com/Badges.csv")
Comments = pd.read_csv("travel_stackexchange_com/Comments.csv")
Users = pd.read_csv("travel_stackexchange_com/Users.csv")
Votes = pd.read_csv("travel_stackexchange_com/Votes.csv")

<br>

## Reference solutions and database

In [3]:
import os, os.path
import sqlite3
import tempfile
# path to database file
baza = os.path.join(tempfile.mkdtemp(), 'example.db')
if os.path.isfile(baza): # if this file already exists...
    os.remove(baza) # ...we will remove it
conn = sqlite3.connect(baza) # create the connection
Badges.to_sql("Badges", conn) # import the data frame into the database
Comments.to_sql("Comments", conn)
Posts.to_sql("Posts", conn)
Users.to_sql("Users", conn)
Votes.to_sql("Votes", conn)

df1 = pd.read_sql_query("""SELECT STRFTIME('%Y', CreationDate) AS Year, COUNT(*) AS TotalNumber
    FROM Posts
    GROUP BY Year""", conn)

df2 = pd.read_sql_query("""SELECT Id, DisplayName, SUM(ViewCount) AS TotalViews
    FROM Users
    JOIN (
    SELECT OwnerUserId, ViewCount FROM Posts WHERE PostTypeId = 1
    ) AS Questions
    ON Users.Id = Questions.OwnerUserId
    GROUP BY Id
    ORDER BY TotalViews DESC
    LIMIT 10""", conn)

df3 = pd.read_sql_query("""SELECT Year, Name, MAX((Count * 1.0) / CountTotal) AS MaxPercentage
    FROM (
    SELECT BadgesNames.Year, BadgesNames.Name, BadgesNames.Count, BadgesYearly.CountTotal
    FROM (
    SELECT Name, COUNT(*) AS Count, STRFTIME('%Y', Badges.Date) AS Year
    FROM Badges
    GROUP BY Name, Year
    ) AS BadgesNames
    JOIN (
    SELECT COUNT(*) AS CountTotal, STRFTIME('%Y', Badges.Date) AS Year
    FROM Badges
    GROUP BY YEAR
    ) AS BadgesYearly
    ON BadgesNames.Year = BadgesYearly.Year
    )
    GROUP BY Year""", conn)

df4 = pd.read_sql_query("""SELECT Title, CommentCount, ViewCount, CommentsTotalScore, DisplayName, Reputation, Location
    FROM (
    SELECT Posts.OwnerUserId, Posts.Title, Posts.CommentCount, Posts.ViewCount,
    CmtTotScr.CommentsTotalScore
    FROM (
    SELECT PostId, SUM(Score) AS CommentsTotalScore
    FROM Comments
    GROUP BY PostId
    ) AS CmtTotScr
    JOIN Posts ON Posts.Id = CmtTotScr.PostId
    WHERE Posts.PostTypeId=1
    ) AS PostsBestComments
    JOIN Users ON PostsBestComments.OwnerUserId = Users.Id
    ORDER BY CommentsTotalScore DESC
    LIMIT 10""", conn)

df5 = pd.read_sql_query("""SELECT Posts.Title, STRFTIME('%Y-%m-%d', Posts.CreationDate) AS Date, VotesByAge.*
    FROM Posts
    JOIN (
    SELECT PostId,
    MAX(CASE WHEN VoteDate = 'before' THEN Total ELSE 0 END) BeforeCOVIDVotes,
    MAX(CASE WHEN VoteDate = 'during' THEN Total ELSE 0 END) DuringCOVIDVotes,
    MAX(CASE WHEN VoteDate = 'after' THEN Total ELSE 0 END) AfterCOVIDVotes,
    SUM(Total) AS Votes
    FROM (
    SELECT PostId,
    CASE STRFTIME('%Y', CreationDate)
    WHEN '2022' THEN 'after'
    WHEN '2021' THEN 'during'
    WHEN '2020' THEN 'during'
    WHEN '2019' THEN 'during'
    ELSE 'before'
    END VoteDate, COUNT(*) AS Total
    FROM Votes
    WHERE VoteTypeId IN (3, 4, 12)
    GROUP BY PostId, VoteDate
    ) AS VotesDates
    GROUP BY VotesDates.PostId
    ) AS VotesByAge ON Posts.Id = VotesByAge.PostId
    WHERE Title NOT IN ('') AND DuringCOVIDVotes > 0
    ORDER BY DuringCOVIDVotes DESC, Votes DESC
    LIMIT 20""", conn)



conn.close()

<br>

## Solution 1
Description: Number of posts for each year


In [4]:
df1_s = solution_1(Posts).applymap(str)
df1_s

,Year,TotalNumber
0,2011,4334
1,2012,6335
2,2013,7614
3,2014,10953
4,2015,14007
5,2016,17087
6,2017,16154
7,2018,15141
8,2019,15499
9,2020,6203


In [5]:
df1

,Year,TotalNumber
0,2011,4334
1,2012,6335
2,2013,7614
3,2014,10953
4,2015,14007
5,2016,17087
6,2017,16154
7,2018,15141
8,2019,15499
9,2020,6203


<br>

## Solution 2
Description: First we select from the users and Id and Display name. From posts, we obtain OwnerUserId ViewCount and PostTypeId We are interested only Posts which are questions (PostTypeId = 1). Merging posts with their owners and finding the top 10 users that are owner of posts with the highest view count.


In [6]:
df2_s = solution_2(Posts, Users)
df2_s 

,Id,DisplayName,TotalViews
0,101,Mark Mayo,2217642.0
1,9009,JonathanReez,2187770.0
2,693,RoflcoptrException,1929754.0
3,140,hippietrail,1927767.0
4,26,Gagravarr,1607181.0
5,3736,nsn,1241921.0
6,396,jrdioko,1016211.0
7,10051,Nean Der Thal,960761.0
8,1893,lambshaanxy,896633.0
9,2509,gerrit,868851.0


In [7]:
df2

,Id,DisplayName,TotalViews
0,101,Mark Mayo,2217642.0
1,9009,JonathanReez,2187770.0
2,693,RoflcoptrException,1929754.0
3,140,hippietrail,1927767.0
4,26,Gagravarr,1607181.0
5,3736,nsn,1241921.0
6,396,jrdioko,1016211.0
7,10051,Nean Der Thal,960761.0
8,1893,lambshaanxy,896633.0
9,2509,gerrit,868851.0


<br>

## Solution 3

Description: We start with counting how many badges of every type were given each year. Moreover, we calculate the total number of badges given each year. In the next step, we checked the popularity of every Badge. In the final result, only the most popular badges of the year are selected.

In [8]:
df3_s = solution_3(Badges)
df3_s 

,Year,Name,MaxPercentage
0,2011,Supporter,0.166925
1,2012,Supporter,0.130064
2,2013,Supporter,0.117536
3,2014,Autobiographer,0.214061
4,2015,Autobiographer,0.147789
5,2016,Student,0.119992
6,2017,Student,0.136280
7,2018,Student,0.133628
8,2019,Student,0.137143
9,2020,Yearling,0.118042


In [9]:
df3

,Year,Name,MaxPercentage
0,2011,Supporter,0.166925
1,2012,Supporter,0.130064
2,2013,Supporter,0.117536
3,2014,Autobiographer,0.214061
4,2015,Autobiographer,0.147789
5,2016,Student,0.119992
6,2017,Student,0.136280
7,2018,Student,0.133628
8,2019,Student,0.137143
9,2020,Yearling,0.118042


<br>

## Solution 4
Description: Firstly we calculate the sum of every post - the score of comments (probable reaction added to comments) is a measure of the popularity of the post. We pick only posts that are questions. Next, we obtain information about the Owners of the post. Finally, we select 10 posts that are the most popular.

In [10]:
df4_s = solution_4(Comments, Posts, Users).fillna('None')
df4_s

,Title,CommentCount,ViewCount,CommentsTotalScore,DisplayName,Reputation,Location
0,"Boss is asking for passport, but it has a stam...",24,61309.0,630,Megha,1679,None
1,How to intentionally get denied entry to the U...,37,72856.0,618,user11743,1997,None
2,Can I wear a bulletproof vest while traveling ...,26,23467.0,403,Ulkoma,9190,"London, United Kingdom"
3,"OK we're all adults here, so really, how on ea...",29,113001.0,388,hippietrail,75994,"Sunshine Coast QLD, Australia"
4,Being separated from one's young children on a...,36,14308.0,358,Emma-louise,611,None
5,Immigration officer that stopped me at the air...,24,22495.0,345,DiegoJ,1273,Buenos Aires
6,How to avoid toddlers on a long-distance plane...,18,29344.0,343,Silver Dragon,943,"San Francisco, United States"
7,Can I fly with a gold bar?,25,55450.0,333,Ulkoma,9190,"London, United Kingdom"
8,How to book a flight if my passport doesn't st...,23,18029.0,330,kukis,894,"Sztokholm, Szwecja"
9,Why don't airlines have backup planes just in ...,26,18809.0,328,clickbait,542,None


In [11]:
df4

,Title,CommentCount,ViewCount,CommentsTotalScore,DisplayName,Reputation,Location
0,"Boss is asking for passport, but it has a stam...",24,61309.0,630,Megha,1679,None
1,How to intentionally get denied entry to the U...,37,72856.0,618,user11743,1997,None
2,Can I wear a bulletproof vest while traveling ...,26,23467.0,403,Ulkoma,9190,"London, United Kingdom"
3,"OK we're all adults here, so really, how on ea...",29,113001.0,388,hippietrail,75994,"Sunshine Coast QLD, Australia"
4,Being separated from one's young children on a...,36,14308.0,358,Emma-louise,611,None
5,Immigration officer that stopped me at the air...,24,22495.0,345,DiegoJ,1273,Buenos Aires
6,How to avoid toddlers on a long-distance plane...,18,29344.0,343,Silver Dragon,943,"San Francisco, United States"
7,Can I fly with a gold bar?,25,55450.0,333,Ulkoma,9190,"London, United Kingdom"
8,How to book a flight if my passport doesn't st...,23,18029.0,330,kukis,894,"Sztokholm, Szwecja"
9,Why don't airlines have backup planes just in ...,26,18809.0,328,clickbait,542,None


<br>

## Solution 5
Description: We select only votes of type 3 (down-vote), 4 (offensive) or 12 (spam) (note: all votes are negative reaction). Depending on the year, votes belong to one of 3 classes (Before covid, during covid, or after covid). Next we summarize Post information showing number of votes before, during and after and the total number of votes. (note: before covid period contains many years so we consider only the most active year of the post) Posts are sorted by DuringCovidVotes and the total number of votes. 20 posts that had the highest number of votes during covid without empty titles. (most negative reaction of post during covid)

In [12]:
df5_s = solution_5(Votes, Posts)
df5_s 

,Title,Date,PostId,BeforeCOVIDVotes,DuringCOVIDVotes,AfterCOVIDVotes,Votes
0,Which countries have still NOT yet reported a ...,2020-03-15,154995,0,23,0,23
1,Why don't hotels offer (at least) 1 kitchen bo...,2019-02-17,132343,0,17,0,17
2,Where can I go to avoid planes overhead?,2019-04-30,137619,0,13,0,13
3,Can male guests ejaculate in hotels' dispenser...,2019-08-29,144746,0,13,0,13
4,Can I make a leisure trip to California (vacat...,2021-01-18,162140,0,13,0,13
5,Why do some countries have inconsistent rules ...,2021-09-30,168959,0,13,0,13
6,What are one's options when facing religious d...,2019-09-24,147417,0,12,0,12
7,"How can you escape, if racists in London, who ...",2020-03-04,154601,0,12,0,12
8,Where's this yellowish building in Texas?,2020-05-21,157337,0,12,0,12
9,How long before boarding gates close are last ...,2021-08-25,168374,0,12,0,12


In [13]:
df5

,Title,Date,PostId,BeforeCOVIDVotes,DuringCOVIDVotes,AfterCOVIDVotes,Votes
0,Which countries have still NOT yet reported a ...,2020-03-15,154995,0,23,0,23
1,Why don't hotels offer (at least) 1 kitchen bo...,2019-02-17,132343,0,17,0,17
2,Where can I go to avoid planes overhead?,2019-04-30,137619,0,13,0,13
3,Can male guests ejaculate in hotels' dispenser...,2019-08-29,144746,0,13,0,13
4,Can I make a leisure trip to California (vacat...,2021-01-18,162140,0,13,0,13
5,Why do some countries have inconsistent rules ...,2021-09-30,168959,0,13,0,13
6,What are one's options when facing religious d...,2019-09-24,147417,0,12,0,12
7,"How can you escape, if racists in London, who ...",2020-03-04,154601,0,12,0,12
8,Where's this yellowish building in Texas?,2020-05-21,157337,0,12,0,12
9,How long before boarding gates close are last ...,2021-08-25,168374,0,12,0,12


<br>

## Comparing equivalence of solutions
Finally, we compare results of sql and pandas function. We need to make sure that column types are the same with use of applymap:


In [14]:
df1.applymap(str).equals(df1_s.applymap(str))

True

In [15]:
df2.applymap(str).equals(df2_s.applymap(str))

True

In [16]:
df3.applymap(str).equals(df3_s.applymap(str))

True

In [17]:
df4.applymap(str).equals(df4_s.applymap(str))

True

In [18]:
df5.applymap(str).equals(df5_s.applymap(str))

True

## Summary
As we can see all solutions written are equivalent. Solutions to the pandas homeworks seems to be similar to data.table package in R from the previous homework. 
<br/>
Note: Comments in .py code are not added because the logic is the same as in the previous homework
